# seq2seq

In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm

sys.path.append(os.path.abspath("modules/"))
sys.path.append(os.path.abspath("fe_modules/"))
sys.path.append(os.path.abspath("seq2seq_modules/"))

os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
tqdm.pandas()

In [1]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from transformers import get_constant_schedule

In [3]:
import importlib

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat
from seq2seq_modules.data import TestDataset
from seq2seq_modules.models import LSTMModel
from seq2seq_modules.weight_initialization import weights_init_uniform_rule
from seq2seq_modules.loops import cross_validation, single_model_training
from seq2seq_modules.utils import age_bucket
from seq2seq_modules.metrics import AGE_METRIC
from seq2seq_modules.utils import fix_random_state

## Read and process

In [4]:
LOCAL_DATA_PATH = './data/'
SEED = 42
fix_random_state(SEED)

In [5]:
ids = ["user_id"]

cat_features = [
    "region_name",
    "city_name",
    "cpe_manufacturer_name",
    "cpe_model_name",
    "url_host",
    "cpe_type_cd",
    "cpe_model_os_type",
    "part_of_day",
    "domain",
    "capital_marker"
]

continous_features = [
    "request_cnt",
    "price",
    "timestamp",
    "relative_timestamp",
    "geo_lat",
    "geo_lon",
    "population",
    "timezone",
    "dist_to_Moscow",
    "dist_to_SaintP",
    "dist_to_Novosibirsk",
    "dist_to_Ekaterinburg",
    "dist_to_Vladivostok",
]

In [6]:
df = pandas_reduce_mem_usage(
    pd.read_parquet("seq2seq_data/stages/stage_2.parquet.gzip")
)
df.head()

Memory usage of dataframe is 26346.12 MB


  0%|          | 0/30 [00:00<?, ?it/s]

Memory usage after optimization is: 26346.12 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,timezone,geo_lat,geo_lon,population,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain
0,21,409,1,589,5788,2,1,20368.0,2,1,...,3,45.040161,38.975964,744933,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761
1,21,409,1,589,12900,2,1,20368.0,2,1,...,3,45.040161,38.975964,744933,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549
2,21,409,1,589,17626,2,1,20368.0,0,1,...,3,45.040161,38.975964,744933,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712
3,21,409,1,589,59366,2,1,20368.0,0,1,...,3,45.040161,38.975964,744933,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712
4,21,409,1,589,59366,2,1,20368.0,0,1,...,3,45.040161,38.975964,744933,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712


In [7]:
needed_ids = pd.read_parquet(
             f'{LOCAL_DATA_PATH}submit_2.pqt'
         )

needed_ids

,user_id
221301,221301
31271,31271
211594,211594
253119,253119
192578,192578
...,...
145307,415284
145308,415285
145309,415286
145312,415306


In [8]:
df = needed_ids.merge(df, how="left", on="user_id")
df.head()

,user_id,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,...,timezone,geo_lat,geo_lon,population,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain
0,221301,44,894,1,593,59366,2,1,6240.0,0,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,712
1,221301,44,894,1,593,160869,2,1,6240.0,2,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,549
2,221301,44,894,1,593,197762,2,1,20196.0,1,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,761
3,221301,44,894,1,593,5790,2,1,6240.0,3,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,761
4,221301,44,894,1,593,68348,2,1,20196.0,2,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,549


In [9]:
LOCAL_DATA_PATH = './data/'
SPLIT_SEED = 42

In [10]:
ids = ["user_id"]

cat_features = [
    "region_name",
    "city_name",
    "cpe_manufacturer_name",
    "cpe_model_name",
    "url_host",
    "cpe_type_cd",
    "cpe_model_os_type",
    "part_of_day",
    "domain",
    "capital_marker"
]

continous_features = [
    "request_cnt",
    "price",
    "timestamp",
    "relative_timestamp",
    "geo_lat",
    "geo_lon",
    "population",
    "timezone",
    "dist_to_Moscow",
    "dist_to_SaintP",
    "dist_to_Novosibirsk",
    "dist_to_Ekaterinburg",
    "dist_to_Vladivostok",
]

In [11]:
dataset = TestDataset(
         df,
         agg_column="user_id", 
         time_column="timestamp",
         cat_features=cat_features,
         cont_features=continous_features,
         max_len=1024,
         padding_side="left",
)
user_ids = dataset.get_agg_col()
dataset

  0%|          | 0/113777491 [00:00<?, ?it/s]

In [12]:
df.head()

,user_id,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,...,timezone,geo_lat,geo_lon,population,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain
0,221301,44,894,1,593,59366,2,1,6240.0,0,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,712
1,221301,44,894,1,593,160869,2,1,6240.0,2,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,549
2,221301,44,894,1,593,197762,2,1,20196.0,1,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,761
3,221301,44,894,1,593,5790,2,1,6240.0,3,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,761
4,221301,44,894,1,593,68348,2,1,20196.0,2,...,5,54.734943,55.957848,1062300,1168.546509,1636.681519,1720.041016,373.426453,5426.129395,549


## Feed to the model

In [13]:
cat_feature_indexes = []
cont_feature_indexes = []
vocab_sizes = {}

for i in tqdm(range(len(cat_features))):
    cat_feature_indexes.append(i)
    vocab_sizes[i] = int(df[cat_features[i]].max() + 1)

for i in tqdm(range(len(continous_features))):
    cont_feature_indexes.append(i)

100%|███████████████████████████████████████| 13/13 [00:00<00:00, 403895.94it/s]


In [14]:
import joblib

MODEL_ZOO = "model_zoo/"

age_embedding_extractor = joblib.load(f"{MODEL_ZOO}/age_baseline.joblib")
is_male_embedding_extractor = joblib.load(f"{MODEL_ZOO}/is_male_baseline.joblib")

In [15]:
age_embeddings_df, age_logits_df = age_embedding_extractor.transform(dataset, user_ids)

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

In [16]:
is_male_embeddings_df, is_male_logits_df = is_male_embedding_extractor.transform(dataset, user_ids)

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

  0%|          | 0/4523 [00:00<?, ?it/s]

In [18]:
is_male_logits_df

,target_feature_0,target_feature_1,user_id
0,0.702871,0.321732,6
1,0.384475,0.637350,7
2,0.743736,0.276358,9
3,0.747826,0.271077,10
4,0.294060,0.723262,11
...,...,...,...
144719,0.469736,0.553725,415306
144720,0.518267,0.506669,415310
144721,0.523229,0.501669,415314
144722,0.524765,0.500138,415315


In [19]:
age_logits_df

,target_feature_0,target_feature_1,target_feature_2,target_feature_3,target_feature_4,target_feature_5,target_feature_6,user_id
0,0.125680,0.191276,0.243778,0.194149,0.107965,0.072448,0.064704,6
1,0.071202,0.115327,0.222334,0.235148,0.158775,0.110904,0.086310,7
2,0.144367,0.415936,0.256378,0.087747,0.039466,0.026817,0.029289,9
3,0.028000,0.163096,0.388326,0.257741,0.099538,0.040792,0.022508,10
4,0.002928,0.001951,0.008651,0.037646,0.147860,0.348237,0.452726,11
...,...,...,...,...,...,...,...,...
144719,0.108397,0.090517,0.131950,0.155867,0.158482,0.175166,0.179620,415306
144720,0.100576,0.077580,0.118733,0.149628,0.159817,0.188231,0.205437,415310
144721,0.290963,0.153670,0.129192,0.129342,0.103447,0.095868,0.097516,415314
144722,0.088087,0.065240,0.106350,0.142428,0.160366,0.201482,0.236047,415315


In [22]:
def merge_predictions(is_male, age):
    is_male_proba = is_male["target_feature_1"].to_numpy()
    age_class = age[[feature for feature in age.columns if "feature" in feature]].to_numpy().argmax(axis=1)
    
    final_df = pd.DataFrame({"user_id": age["user_id"], "age": age_class, "is_male": is_male_proba})
    return final_df

In [23]:
final_df = merge_predictions(is_male_logits_df, age_logits_df)
final_df

,user_id,age,is_male
0,6,2,0.321732
1,7,3,0.637350
2,9,1,0.276358
3,10,2,0.271077
4,11,6,0.723262
...,...,...,...
144719,415306,6,0.553725
144720,415310,6,0.506669
144721,415314,0,0.501669
144722,415315,6,0.500138


In [24]:
final_df.to_csv("submission_1.csv", index=False)